## 1. Подсчет пропущенных значений
<p>Производство спортивной одежды и одежды для отдыха — это огромная индустрия стоимостью около <a href="https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/">193 миллиарда долларов США в 2022 году</a> и прогнозируемый сильный рост в течение следующего десятилетия. </p>
<p>В этом блокноте мы выполняли роль аналитика по продукции интернет-компании, производящей спортивную одежду. Компания особенно заинтересована в том, как она может увеличить доходы. Мы углубимся в данные о продуктах, такие как цены, обзоры, описания и рейтинги, а также доходы и посещаемость веб-сайта, чтобы подготовить рекомендации для отделов маркетинга и продаж. </p>
<p>Предоставленная нам база данных под названием <code>sports</code> содержит пять таблиц, где <code>product_id</code> является первичным ключом для всех них: </p>
<h3 id="info"><code>info</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_name</code></td>
<td><code>varchar</code></td>
<td>Name of the product</td>
</tr>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>description</code></td>
<td><code>varchar</code></td>
<td>Description of the product</td>
</tr>
</tbody>
</table>
<h3 id="finance"><code>finance</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>listing_price</code></td>
<td><code>float</code></td>
<td>Listing price for product</td>
</tr>
<tr>
<td><code>sale_price</code></td>
<td><code>float</code></td>
<td>Price of the product when on sale</td>
</tr>
<tr>
<td><code>discount</code></td>
<td><code>float</code></td>
<td>Discount, as a decimal, applied to the sale price</td>
</tr>
<tr>
<td><code>revenue</code></td>
<td><code>float</code></td>
<td>Amount of revenue generated by each product, in US dollars</td>
</tr>
</tbody>
</table>
<h3 id="reviews"><code>reviews</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_name</code></td>
<td><code>varchar</code></td>
<td>Name of the product</td>
</tr>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>rating</code></td>
<td><code>float</code></td>
<td>Product rating, scored from <code>1.0</code> to <code>5.0</code></td>
</tr>
<tr>
<td><code>reviews</code></td>
<td><code>float</code></td>
<td>Number of reviews for the product</td>
</tr>
</tbody>
</table>
<h3 id="traffic"><code>traffic</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>last_visited</code></td>
<td><code>timestamp</code></td>
<td>Date and time the product was last viewed on the website</td>
</tr>
</tbody>
</table>
<h3 id="brands"><code>brands</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>brand</code></td>
<td><code>varchar</code></td>
<td>Brand of the product</td>
</tr>
</tbody>
</table>
<p>Мы будем иметь дело с недостающими данными, а также с числовыми, строковыми типами данных и метками времени, чтобы получить представление о продуктах в интернет-магазине. Начнем с того, что выясним, насколько полны данные.</p>

In [2]:
%%sql
postgresql:///sports
    
-- Подсчет всех столбцов как total_rows
-- Подсчет количества непропущенных записей для 'description', 'listing_price' и 'last_visited'.
-- Объединение 'info', 'finance' и 'traffic'

SELECT COUNT(*) AS total_rows, 
    COUNT(i.description) AS count_description, 
    COUNT(f.listing_price) AS count_listing_price, 
    COUNT(t.last_visited) AS count_last_visited 
FROM info AS i
INNER JOIN finance AS f
    ON i.product_id = f.product_id
INNER JOIN traffic AS t
    ON t.product_id = f.product_id;

1 rows affected.


total_rows,count_description,count_listing_price,count_last_visited
3179,3117,3120,2928


## 2. Цены Nike против Adidas
<p>Мы видим, что база данных содержит всего 3179 товаров. Из просмотренных нами столбцов только один &mdash; <code>last_visited</code> &mdash; отсутствует более пяти процентов его значений. Теперь обратим внимание на ценообразование. </p>
<p>Чем отличаются цены на продукцию Nike и Adidas? Ответ на этот вопрос может помочь нам составить представление об ассортименте акций компании и потребительском рынке. Мы выполним запрос, чтобы получить распределение <code>listing_price</code> и количества для каждой цены, сгруппированных по <code>бренду</code>. </p>

In [4]:
%%sql
    
-- Выбор бренда, listing_price как целое число и подсчет всех продуктов в разделе «финансы».
-- Присоединение 'brands' к 'finance' в 'product_id'
-- Фильтрация товаров, у которых listing_price больше нуля.
-- Объединение результатов по 'brand' и listing_price и сортировка результатов по listing_price в порядке убывания.

SELECT b.brand, f.listing_price::integer, COUNT(f.*)
FROM finance AS f
INNER JOIN brands AS b 
    ON f.product_id = b.product_id
WHERE listing_price > 0
GROUP BY b.brand, f.listing_price
ORDER BY listing_price DESC;

 * postgresql:///sports
77 rows affected.


brand,listing_price,count
Adidas,300,2
Adidas,280,4
Adidas,240,5
Adidas,230,8
Adidas,220,11
Nike,200,1
Adidas,200,8
Nike,190,2
Adidas,190,7
Nike,180,4


## 3. Маркировка ценовых диапазонов
<p>Оказывается, в нашей базе данных 77 уникальных цен на продукты, что затрудняет анализ результатов нашего последнего запроса. </p>
<p>Давайте продолжим наш предыдущий запрос, назначив ярлыки разным ценовым диапазонам, группируя их по <code>'brand'</code> и <code>'label'</code>. Мы также укажем общий <code>'revenue'</code> для каждого ценового диапазона и <code>'brand'</code>. </p>

In [6]:
%%sql

-- Выбор бренда, количество всех продуктов в таблице 'finance' и общий доход.
-- Создание четырех ярлыков для продуктов в зависимости от их ценового диапазона с псевдонимом price_category.
-- Присоединение брендов к финансированию по Product_id и фильтрация продуктов, в которых отсутствует значение бренда.
-- Группировка результатов по 'brand' и price_category, сортировка по total_revenue.

SELECT b.brand, COUNT(f.*), SUM(f.revenue) as total_revenue,
CASE WHEN f.listing_price < 42 THEN 'Budget'
    WHEN f.listing_price >= 42 AND f.listing_price < 74 THEN 'Average'
    WHEN f.listing_price >= 74 AND f.listing_price < 129 THEN 'Expensive'
    ELSE 'Elite' END AS price_category
FROM finance AS f
INNER JOIN brands AS b 
    ON f.product_id = b.product_id
WHERE b.brand IS NOT NULL
GROUP BY b.brand, price_category
ORDER BY total_revenue DESC;

 * postgresql:///sports
8 rows affected.


brand,count,total_revenue,price_category
Adidas,849,4626980.069999999,Expensive
Adidas,1060,3233661.060000001,Average
Adidas,307,3014316.8299999987,Elite
Adidas,359,651661.1200000002,Budget
Nike,357,595341.0199999992,Budget
Nike,82,128475.59000000003,Elite
Nike,90,71843.15000000004,Expensive
Nike,16,6623.5,Average


## 4. Средняя скидка по бренду
<p>Интересно, что группировка товаров по брендам и ценовым диапазонам позволяет нам увидеть, что товары Adidas приносят больший общий доход независимо от ценовой категории. В частности, <code>"Элитные"</code> продукты Adidas по цене 129 долларов США и выше обычно приносят самый высокий доход, поэтому компания потенциально может увеличить доход, переместив свои запасы на большую долю этих продуктов.</p>
<p>Обратите внимание, что до сих пор мы рассматривали <code>listing_price</code>. <code>listing_price</code> может не совпадать с ценой, по которой в конечном итоге будет продан продукт. Чтобы лучше понять <code>'revenue'</code>, давайте взглянем на <code>'discount'</code>, которая представляет собой процентное снижение <code>listing_price</code> при фактической продаже продукта. Мы хотели бы знать, существует ли разница в сумме <code>'discount'</code>, предлагаемой между брендами, поскольку это может влиять на <code>'revenue'</code>.</p>

In [8]:
%%sql
    
-- Выбор бренда и average_discount в процентах
-- Присоединение брендов к финансированию на product_id
-- Агрегирование по брендам
-- Фильтрация продуктов без пропущенных значений бренда

SELECT b.brand, AVG(f.discount) * 100 AS average_discount
FROM brands AS b
INNER JOIN finance AS f 
    ON b.product_id = f.product_id
GROUP BY b.brand
HAVING b.brand IS NOT NULL
ORDER BY average_discount;

 * postgresql:///sports
2 rows affected.


brand,average_discount
Nike,0.0
Adidas,33.452427184465606


## 5. Корреляция между доходом и отзывами
<p>Как ни странно, на продукцию Nike скидки не предоставляются. Для сравнения, продукция Adidas не только приносит наибольшую прибыль, но и продается с большими скидками. </p>
<p>Чтобы еще больше увеличить выручку, компания могла бы попытаться уменьшить размер скидок, предлагаемых на продукцию Adidas, и отслеживать объем продаж, чтобы увидеть, остается ли он стабильным. В качестве альтернативы компания может попытаться предложить небольшую скидку на продукцию Nike. Это снизит средний доход от этих продуктов, но может увеличить доход в целом, если произойдет увеличение объема продаваемой продукции Nike. </p>
<p>Теперь выясним, существуют ли связи между столбцами в нашей базе данных. Мы проверим силу и направление корреляции между <code>'revenue'</code> и <code>'reviews'</code>. </p>

In [10]:
%%sql

-- Расчет корреляции между отзывами и доходом как review_revenue_corr.
-- Объединение таблиц 'reviews' и 'finance' по product_id.

SELECT corr(r.reviews, f.revenue) AS review_revenue_corr
FROM reviews AS r
INNER JOIN finance AS f 
    ON r.product_id = f.product_id;

 * postgresql:///sports
1 rows affected.


review_revenue_corr
0.6518512283481301


## 6. Оценки и отзывы по длине описания товара
<p>Интересно, что существует сильная положительная корреляция между доходом и отзывами. Это означает, что потенциально, если мы сможем получить больше отзывов на веб-сайте компании, это может увеличить продажи товаров с большим количеством отзывов. </p>
<p>Возможно, длина <code>описания</code> продукта может повлиять на его <code>рейтинг</code> и <code>отзывы</code> &mdash; если да, то компания может разработать рекомендации по содержанию для размещения продуктов на своем веб-сайте и проверить, влияет ли это на <code>доход</code>.</p>

In [12]:
%%sql

-- Вычисление description_length
-- Преобразование рейтинга в числовой тип данных и вычисление average_rating
-- Объединение информации с отзывами по product_id и группировка результатов по description_length
-- Фильтрация товаров без пропущенных значений описания и сортировка результатов по description_length

SELECT TRUNC(LENGTH(i.description), -2) AS description_length,
    ROUND(AVG(r.rating::numeric), 2) AS average_rating
FROM info AS i
INNER JOIN reviews AS r 
    ON i.product_id = r.product_id
WHERE i.description IS NOT NULL
GROUP BY description_length
ORDER BY description_length;

 * postgresql:///sports
7 rows affected.


description_length,average_rating
0,1.87
100,3.21
200,3.27
300,3.29
400,3.32
500,3.12
600,3.65


## 7. Отзывы по месяцам и брендам
<p>К сожалению, не существует четкой зависимости между длиной <code>описания</code> продукта и его <code>рейтингом</code>.</p>
<p>Поскольку мы знаем, что существует корреляция между <code>отзывами</code> и <code>доходом</code>, компания может использовать один из подходов — проводить эксперименты с различными процессами продаж, стимулируя больше отзывов от клиентов об их покупках, например: предлагая небольшую скидку на будущие покупки. </p>
<p>Давайте посмотрим на количество <code>отзывов</code> по месяцам, чтобы увидеть, есть ли какие-либо тенденции или пробелы, которые мы можем использовать.</p>

In [14]:
%%sql

-- Выбор бренда, месяца с момента last_visited и количества всех продуктов в обзорах, имеющих псевдоним num_reviews
-- Объединение трафика с отзывами и брендами по product_id
-- Группировка по бренду и месяцу, фильтрация пропущенных значений по бренду и месяцу
-- Упорядочение результатов по бренду и месяцу

SELECT b.brand, DATE_PART('month', t.last_visited) AS month, COUNT(r.*) AS num_reviews
FROM brands AS b
INNER JOIN traffic AS t 
    ON b.product_id = t.product_id
INNER JOIN reviews AS r 
    ON t.product_id = r.product_id
GROUP BY b.brand, month
HAVING b.brand IS NOT NULL
    AND DATE_PART('month', t.last_visited) IS NOT NULL
ORDER BY b.brand, month;

 * postgresql:///sports
24 rows affected.


brand,month,num_reviews
Adidas,1.0,253
Adidas,2.0,272
Adidas,3.0,269
Adidas,4.0,180
Adidas,5.0,172
Adidas,6.0,159
Adidas,7.0,170
Adidas,8.0,189
Adidas,9.0,181
Adidas,10.0,192


## 8. Производительность обувной продукции
<p>Похоже, что количество отзывов о продуктах самое большое в первом квартале календарного года, поэтому есть возможность провести эксперименты, направленные на увеличение количества отзывов в остальные девять месяцев.</p>
<p>До сих пор мы в первую очередь анализировали продукцию Adidas и Nike. Теперь давайте переключим внимание на тип продаваемой продукции. Поскольку меток для типа продукта нет, мы создадим общее табличное выражение (CTE), которое фильтрует <code>'description'</code> по ключевым словам, а затем использует результаты, чтобы узнать, какая часть запасов компании состоит из обуви. продукты и средний <code>доход</code>, полученный от этих товаров.</p>

In [16]:
%%sql

-- Создание CTE для обуви, содержащего описание и доход
-- Фильтрация обуви для товаров, описание которых содержит %shoe%, %trainer или %foot%
-- Также фильтрация продуктов, у которых нет пропущенных значений в описании
-- Расчет количества продуктов и среднего дохода от обувной продукции

WITH footwear AS
(
    SELECT i.description, f.revenue
    FROM info AS i
    INNER JOIN finance AS f 
        ON i.product_id = f.product_id
    WHERE i.description ILIKE '%shoe%'
        OR i.description ILIKE '%trainer%'
        OR i.description ILIKE '%foot%'
        AND i.description IS NOT NULL
)

SELECT COUNT(*) AS num_footwear_products, 
    percentile_disc(0.5) WITHIN GROUP (ORDER BY revenue) AS median_footwear_revenue
FROM footwear;

 * postgresql:///sports
1 rows affected.


num_footwear_products,median_footwear_revenue
2700,3118.36


## 9. Производительность одежды
<p>Напомним, что на первом этапе мы обнаружили, что у 3117 продуктов нет пропущенных значений для <code>'description'</code>. Из них 2700 приходится на обувную продукцию, что составляет около 85% акций компании. Они также генерируют средний доход более 3000 долларов.</p>
<p>Это интересно, но у нас нет точки зрения на то, является ли <code>median_revenue</code> обуви хорошим или плохим по сравнению с другими продуктами. Итак, на последнем этапе давайте рассмотрим, чем это отличается от одежды. Мы будем повторно использовать <code>'footwear'</code>, добавив после этого фильтр для подсчета количества продуктов и <code>median_revenue</code> продуктов, которые не входят в <code>'footwear'</code>.</p>

In [18]:
%%sql

-- Копирование CTE 'footwear' из предыдущей задачи
-- Расчет количества продуктов в разделе 'info' и 'median revenue' в разделе 'finance'
-- Внутреннее объединение 'info' с 'finance' в product_id
-- Фильтрация выбора товаров с описанием не в разделе 'footwear'

WITH footwear AS
(
    SELECT i.description, f.revenue
    FROM info AS i
    INNER JOIN finance AS f 
        ON i.product_id = f.product_id
    WHERE i.description ILIKE '%shoe%'
        OR i.description ILIKE '%trainer%'
        OR i.description ILIKE '%foot%'
        AND i.description IS NOT NULL
)

SELECT COUNT(i.*) AS num_clothing_products, 
    percentile_disc(0.5) WITHIN GROUP (ORDER BY f.revenue) AS median_clothing_revenue
FROM info AS i
INNER JOIN finance AS f on i.product_id = f.product_id
WHERE i.description NOT IN (SELECT description FROM footwear);

 * postgresql:///sports
1 rows affected.


num_clothing_products,median_clothing_revenue
417,503.82
